## Step 1: Creating Tokens

In [58]:
with open("wharton_verdict.txt","r", encoding = "utf-8") as f:
    raw_text = f.read()
print(raw_text[:99])
print(f'total number of characters are : {len(raw_text)}')

The verdict
Edith wharton

I had always thought Jack Gisburn rather a cheap genius--though a good f
total number of characters are : 20415


Our goal is to tokenize this 20,479-character short story into individual words and special
characters that we can then turn into embeddings for LLM training  

In [59]:
import re

text = "hello, world. This is a text file!!!. It contains some text."
result = re.split(r'(\s)', text) ## \s splits wherever thiere is a whitespace

print(result)

['hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'text', ' ', 'file!!!.', ' ', 'It', ' ', 'contains', ' ', 'some', ' ', 'text.']


The result is a list of individual words, whitespaces, and punctuation characters


Right now the commas and fullstops are part of the words as they dont have white spaces

In [60]:
result = re.split(r'([,.]|\s)', text)
print(result)

['hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'text', ' ', 'file!!!', '.', '', ' ', 'It', ' ', 'contains', ' ', 'some', ' ', 'text', '.', '']


We can see that the words and punctuation characters are now separate list entries just as
we wanted

A small remaining issue is that the list still includes whitespace characters. Optionally, we
can remove these redundant characters safely as follows

In [61]:
result = [item for item in result if item.strip()]
print(result)

['hello', ',', 'world', '.', 'This', 'is', 'a', 'text', 'file!!!', '.', 'It', 'contains', 'some', 'text', '.']



REMOVING WHITESPACES OR NOT


When developing a simple tokenizer, whether we should encode whitespaces as
separate characters or just remove them depends on our application and its
requirements. Removing whitespaces reduces the memory and computing
requirements. However, keeping whitespaces can be useful if we train models that
are sensitive to the exact structure of the text (for example, Python code, which is
sensitive to indentation and spacing). Here, we remove whitespaces for simplicity
and brevity of the tokenized outputs. Later, we will switch to a tokenization scheme
that includes whitespaces.

In [62]:
result = re.split(r'([,.:;"!?()\'_]|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['hello', ',', 'world', '.', 'This', 'is', 'a', 'text', 'file', '!', '!', '!', '.', 'It', 'contains', 'some', 'text', '.']


Now we apply this basic tokenizer to edith whartons short story

In [63]:
def basic_tokenizer(text):
    result = re.split(r'([,.:;"!?()\'_]|--|\s)', text)
    result = [item for item in result if item.strip()]
    print(result[:50])
    return result

In [113]:
pre_processed = basic_tokenizer(raw_text)


['The', 'verdict', 'Edith', 'wharton', 'I', 'had', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow']


In [65]:
print(f'the total numeber of words : {len(pre_processed)}')

the total numeber of words : 4650


## Step 2: Creating Token IDs

In the previous section, we tokenized Edith Wharton's short story and assigned it to a
Python variable called preprocessed. Let's now create a list of all unique tokens and sort
them alphabetically to determine the vocabulary size

In [109]:
all_words = sorted(set(pre_processed))
vocab_size = len(all_words)
print(all_words[:20])
print(f'the total number of unique words : {vocab_size}')

['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'AM', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At']
the total number of unique words : 1150


In [112]:
vocab = {token:integer for integer, token in enumerate(all_words)}
print(vocab['and'])
print(vocab['the'])

169
1006


As we can see, based on the output above, the dictionary contains individual tokens
associated with unique integer labels. 

Later in this book, when we want to convert the outputs of an LLM from numbers back into
text, we also need a way to turn token IDs into text. 

For this, we can create an inverse
version of the vocabulary that maps token IDs back to corresponding text tokens.

Let's implement a complete tokenizer class in Python.

The class will have an encode method that splits
text into tokens and carries out the string-to-integer mapping to produce token IDs via the
vocabulary. 

In addition, we implement a decode method that carries out the reverse
integer-to-string mapping to convert the token IDs back into text.


<div class="alert alert-block alert-info">
    
Step 1: Store the vocabulary as a class attribute for access in the encode and decode methods
    
Step 2: Create an inverse vocabulary that maps token IDs back to the original text tokens

Step 3: Process input text into token IDs

Step 4: Convert token IDs back into text

Step 5: Replace spaces before the specified punctuation

</div>



In [68]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int =vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        pre_processed = re.split(r'([,.:;"!?()\'_]|--|\s)', text)   
        pre_processed = [item for item in pre_processed if item.strip()]
        ids = [self.str_to_int[word] for word in pre_processed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])   
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)#remove unnecessary spaces before punctuation marks in the string text,   r'\1'This tells Python to replace the entire match (space + punctuation) with just the punctuation mark (i.e., what's in group 1), removing the space.
        return text
        



In [69]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[61, 2, 868, 1006, 616, 547, 764, 5, 1146, 611, 5, 1, 74, 7, 41, 869, 1128, 772, 812, 7]


In [70]:
tokenizer.decode(ids)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

<div class="alert alert-block alert-success">

So far, so good. We implemented a tokenizer capable of tokenizing and de-tokenizing
text based on a snippet from the training set. 

Let's now apply it to a new text sample that
is not contained in the training set:
</div>

In [71]:
text = "hello, world. This is a text file!!!. It contains some text."
print(tokenizer.encode(text))

KeyError: 'hello'

<div class="alert alert-block alert-warning">
    
The problem is that the word "Hello" was not used in the The Verdict short story. 

Hence, it
is not contained in the vocabulary. 

This highlights the need to consider large and diverse
training sets to extend the vocabulary when working on LLMs.

</div>

In the previous section, we implemented a simple tokenizer and applied it to a passage
from the training set. 

In this section, we will modify this tokenizer to handle unknown
words.


In particular, we will modify the vocabulary and tokenizer we implemented in the
previous section, SimpleTokenizerV2, to support two new tokens, <|unk|> and
<|endoftext|>


In [72]:
all_tokens = sorted(list(set(pre_processed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}


In [73]:
len(vocab.items())

1152

In [74]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1147)
('your', 1148)
('yourself', 1149)
('<|endoftext|>', 1150)
('<|unk|>', 1151)


In [75]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        pre_processed = re.split(r'([,.:;"!?()\'_]|--|\s)', text)   
        pre_processed = [item for item in pre_processed if item.strip()]
        pre_processed = [item if item in self.str_to_int else "<|unk|>" for item in pre_processed]

        ids = [self.str_to_int[s] for s in pre_processed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [76]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "hello, do you like tea and cars?"
text2 = "in the sunlit terraces of the palace."

text = "<|endoftext|> ".join([text1, text2])
print(text)

hello, do you like tea and cars?<|endoftext|> in the sunlit terraces of the palace.


In [77]:
tokenizer.encode(text)


[1151,
 5,
 369,
 1146,
 643,
 993,
 169,
 1151,
 10,
 1150,
 583,
 1006,
 974,
 1002,
 739,
 1006,
 1151,
 7]

In [78]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea and <|unk|>? <|endoftext|> in the sunlit terraces of the <|unk|>.'

<div class="alert alert-block alert-warning">

So far, we have discussed tokenization as an essential step in processing text as input to
LLMs. Depending on the LLM, some researchers also consider additional special tokens such
as the following:

[BOS] (beginning of sequence): This token marks the start of a text. It
signifies to the LLM where a piece of content begins.

[EOS] (end of sequence): This token is positioned at the end of a text,
and is especially useful when concatenating multiple unrelated texts,
similar to <|endoftext|>. For instance, when combining two different
Wikipedia articles or books, the [EOS] token indicates where one article
ends and the next one begins.

[PAD] (padding): When training LLMs with batch sizes larger than one,
the batch might contain texts of varying lengths. To ensure all texts have
the same length, the shorter texts are extended or "padded" using the
[PAD] token, up to the length of the longest text in the batch.

</div>


<div class="alert alert-block alert-warning">

Note that the tokenizer used for GPT models does not need any of these tokens mentioned
above but only uses an <|endoftext|> token for simplicity

</div>

<div class="alert alert-block alert-warning">

the tokenizer used for GPT models also doesn't use an <|unk|> token for outof-vocabulary words. Instead, GPT models use a byte pair encoding tokenizer, which breaks
down words into subword units
</div>

### BYTE PAIR ENCODING (subword based)


**BPE Tokenizer**

In [79]:
! pip3 install tiktoken 

In [80]:
import tiktoken

In [81]:
tokenizer = tiktoken.get_encoding("gpt2")

In [82]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special= {"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [83]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


data sampling with sliding window

![Screenshot](images/screenshot1.png)


In [84]:
with open("wharton_verdict.txt", "r", encoding= "utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))


5317


In [106]:
enc_sample = enc_text[50:100]
print(enc_sample)
dec_sample = tokenizer.decode(enc_sample)
print(dec_sample)

[18108, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11, 290, 4920, 2241, 287, 257, 4489, 64, 198, 261, 262, 34686, 41976, 13, 357, 10915, 314, 2138, 1807, 340, 561, 423, 587, 10598, 393, 28537, 2014, 198, 1, 464, 6001, 286, 465, 13476, 1, 438, 5562, 373, 644, 262, 1466]
had dropped his painting, married a rich widow, and established himself in a villa
on the Riviera. (Though I rather thought it would have been Rome or Florence.)
"The height of his glory"--that was what the women


In [86]:
context_size = 5
x = enc_sample[:context_size]
y = enc_sample[1:context_size +1]

print(f"x: {x}")
print(f"y:      {y}")

x: [18108, 5710, 465, 12036, 11]
y:      [5710, 465, 12036, 11, 6405]


this x and y represent 4 prediction tasks

x1 -> y1

x1x2 -> y2

x1x2x3 -> y3

x1x2x3x4 -> y4

unlike regression tasks where one input output pair represetns one prediction task


here one input output pair represent multiple prediction tasks as set by the context length

In [87]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "------>", desired)


[18108] ------> 5710
[18108, 5710] ------> 465
[18108, 5710, 465] ------> 12036
[18108, 5710, 465, 12036] ------> 11
[18108, 5710, 465, 12036, 11] ------> 6405


x: [18108, 5710, 465, 12036, 11]
y:      [5710, 465, 12036, 11, 6405]

the above output can be derived from this


In [88]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

had ---->  dropped
had dropped ---->  his
had dropped his ---->  painting
had dropped his painting ----> ,
had dropped his painting, ---->  married


### IMPLEMENTING A DATA LOADER

<div class="alert alert-block alert-warning">

There's only one more task before we can turn the tokens into embeddings:implementing an efficient data loader that
iterates over the input dataset and returns the inputs and targets as PyTorch tensors, which
can be thought of as multidimensional arrays.
    
</div>

<div class="alert alert-block alert-warning">

There's only one more task before we can turn the tokens into embeddings:implementing an efficient data loader that
iterates over the input dataset and returns the inputs and targets as PyTorch tensors, which
can be thought of as multidimensional arrays.
    
</div>

<div class="alert alert-block alert-success">
For the efficient data loader implementation, we will use PyTorch's built-in Dataset and
DataLoader classes.</div>

<div class="alert alert-block alert-info">
    
Step 1: Tokenize the entire text
    
Step 2: Use a sliding window to chunk the book into overlapping sequences of max_length

Step 3: Return the total number of rows in the dataset

Step 4: Return a single row from the dataset
</div>

![Screenshot](images/screenshot2.png)


here the input-output target pairs are x1 -> y1, x2 -> ans so on Xn -> Yn

each input output pair has multiple prediction tasks dependent on the context window

In [96]:
from torch.utils.data import Dataset, DataLoader
import torch
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        #using sliding window to ceate input and target pairs
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i+1: max_length + 1 + i]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):

        return self.input_ids[idx], self.target_ids[idx]

**Example**

We process token_ids in chunks of 5 tokens (max_length = 5) and move forward by 3 tokens each time (stride = 3).

Window 1
i = 0

input_chunk = token_ids[0:5] = [101, 102, 103, 104, 105]

target_chunk = token_ids[1:6] = [102, 103, 104, 105, 106]

Window 2
i = 3

input_chunk = token_ids[3:8] = [104, 105, 106, 107, 108]

target_chunk = token_ids[4:9] = [105, 106, 107, 108, 109]



https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [97]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

<div class="alert alert-block alert-success">
    
Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4, 

This will develop an intuition of how the GPTDatasetV1 class and the
create_dataloader_v1 function work together: </div>

In [98]:
with open("wharton_verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

In [104]:
dataloader = create_dataloader_v1(raw_text, batch_size = 1, max_length = 4, stride = 1, shuffle = False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  464, 15593,   198,  7407]]), tensor([[15593,   198,  7407,   342]])]


play with strides and batch size to get better understanding also learn how the dataloader and datasets libraries work.

https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbDRSY2h3V3lGNUZFWFg0bmtRc3h6QXJTYldaUXxBQ3Jtc0trNlB4V2I5UWxIVDd6VXNEZnNIMkpLLVUxN2hualRyeUM4OS1xUlNjeXBPdXN1bE5xT1ZIMEdFY0U1dGxZdUp3Qzh5Vk8tX054VEFPZkJlZWlfR0RlOWFseVJEUFF3TnpWZHd2QVRwUGpVSnVVTjE1QQ&q=https%3A%2F%2Fpytorch.org%2Ftutorials%2Fbeginner%2Fbasics%2Fdata_tutorial.html&v=iQZFH8dr2yI



https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqa2JWQXNUd1lEdkV2ckFrWHV5R0Frc29fUUdEQXxBQ3Jtc0trZTJ4bjRQWU1JSkFTMEotaUhpWFVDZ2lXQTY3aUw1ZVBmV2NvTU9Fa24tUk5mMDVjQ0ZNbVoxNmljeGM2T1V0dy1aanJmd2VLem9RTS1HeF9EVlZhTjBqanhocU1MTU9XanU0NDJJZ0hxeWpaR3dFZw&q=https%3A%2F%2Fpytorch.org%2Fdocs%2Fstable%2Fdata.html&v=iQZFH8dr2yI

<div class="alert alert-block alert-warning">

The first_batch variable contains two tensors: the first tensor stores the input token IDs,
and the second tensor stores the target token IDs. 

Since the max_length is set to 4, each of the two tensors contains 4 token IDs. 

Note that an input size of 4 is relatively small and only chosen for illustration purposes. It is common to train LLMs with input sizes of at least
256.
    
</div>

<div class="alert alert-block alert-warning">

Batch sizes of 1, such as we have sampled from the data loader so far, are useful for
illustration purposes. 
                                                                                 
If you have previous experience with deep learning, you may know
that small batch sizes require less memory during training but lead to more noisy model
updates.

Just like in regular deep learning, the batch size is a trade-off and hyperparameter
to experiment with when training LLMs.
    
</div>